In [189]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import *
from sympy import *

## The Triad Method.

In order to acquire the desired directional cosine matrix, $[BN]$ an intermediary triad frame $V_i$ is introduced. This frame can be expressed as components of either the body $B$ or inertial $N$ frames.  

The inertial frame triad vectors:

$$
{}^N\hat{v_1} = {}^N\hat{s}
$$

$$
{}^N\hat{v_2} = \frac{({}^N\hat{s}) \times ({}^N\hat{m})}{|({}^N\hat{s}) \times ({}^N\hat{m})|}
$$

$$
{}^N\hat{v_3} = ({}^N\hat{v_1}) \times ({}^N\hat{v_2})
$$

The body frame triad vectors:

$$
{}^B\hat{v_1} = {}^B\hat{s}
$$

$$
{}^B\hat{v_2} = \frac{({}^B\hat{s}) \times ({}^B\hat{m})}{|({}^B\hat{s}) \times ({}^B\hat{m})|}
$$

$$
{}^B\hat{v_3} = ({}^B\hat{v_1}) \times ({}^B\hat{v_2})
$$

Thus one finds the estimated DCM $[\bar{B}N]$ via:




$$
[\bar{B}T] = [{}^B\hat{v_1}, {}^B\hat{v_2}, {}^B\hat{v_3}]
$$

$$
[NT] = [{}^N\hat{v_1}, {}^N\hat{v_2}, {}^N\hat{v_3}]
$$

$$
[\bar{B}N] = [\bar{B}T][NT]^T
$$

For example: A spacecraft has two attitude sensors, sensing two unit vector directions, $v_i$, $i \in 1,2$ with $v_1$ being the more accurate sensor (i.e: Configure $t_1$ and $v_1$ to be co-linear).  

At an instant in time, the two vectors measured by the sensors have the body frame components:

$
{}^B\hat{v_1} = 
\left(\begin{array}{cc} 
0.8273 \\
0.5541 \\
-0.0920
\end{array}\right)
$
$
{}^B\hat{v_2} = 
\left(\begin{array}{cc} 
-0.8285 \\
0.5522 \\
-0.0955
\end{array}\right)
$

And simultaneously, the four vectors are measured to have the inertial frame components:

$
{}^N\hat{v_1} = 
\left(\begin{array}{cc} 
-0.1517 \\
-0.9669\\
-0.2050
\end{array}\right)
$
$
{}^N\hat{v_2} = 
\left(\begin{array}{cc} 
-0.8393 \\
0.4494 \\
-0.3044
\end{array}\right)
$

Using the Triad method to find $[\bar{B}N]$:

In [35]:
B_v1 = np.array([0.8273, 0.5541, -0.0920],  dtype=np.float64)
B_v2 = np.array([-0.8285, 0.5522, -0.0955], dtype=np.float64)

N_v1 = np.array([-0.1517, -0.9669, 0.2050], dtype=np.float64)
N_v2 = np.array([-0.8393, 0.4494, -0.3044], dtype=np.float64)

In [36]:
def Triad_Method(a, b):
    
    axb = np.cross(a, b)
    axb_norm = np.linalg.norm(axb)
    
    v_1 = a
    v_2 = axb/axb_norm
    v_3 = np.cross(v_1, v_2)
    
    C = np.array([v_1, v_2, v_3])
    
    return C.T

In [37]:
BT = Triad_Method(B_v1, B_v2)
NT = Triad_Method(N_v1, N_v2)

In [38]:
#Solving for BN via the property: [BN] = [BT][NT]^T
BN = np.dot(BT, np.linalg.inv(NT))

In [39]:
print(BN)

[[ 0.41555513 -0.85508344  0.31004653]
 [-0.83392479 -0.49427207 -0.24545395]
 [ 0.36313468 -0.15655987 -0.91848883]]


The estimation error of $[\bar{BN}]$ can be found by computing the difference in both angles of $[\bar{B}N]$ and $[BN]$ corresponding to the principal rotation vector:

In [40]:
def PRV_Angle(C):
    return np.arccos(0.5 * (C.trace() - 1))
    

In [41]:
B_N = np.array([[ 0.969846,  0.17101, 0.173648],
                [-0.200706,  0.96461,  0.17101],
                [-0.138258, -0.20076, 0.969846]],)

BN = np.array([[ 0.963592,  0.187303, 0.190809],
               [-0.223042,  0.956645, 0.187303],
               [-0.147454, -0.223042, 0.963592]])

a_B_N = PRV_Angle(B_N)
a_BN = PRV_Angle(BN)

print('Error (degrees):', np.rad2deg(a_BN - a_B_N))

Error (degrees): 1.8284522666836571


## Devenport's q-Method.

The goal is to minimize the cost function $J$ where, 
$$
J = \sum_{k = 1}^N w_k(1 - {}^B\hat{v}_k^T[\bar{B}N]{}^N\hat{v}_k)
$$

In order to do this, we extract function $g$ and find its extremum (maximum), where, 
$$
g = -\sum_{k = 1}^N w_k{}^B\hat{v}_k^T[\bar{B}N]{}^N\hat{v}_k
$$

$g$ can be re-expressed in terms of its quaternion components using a $4 x 4$ matrix $[K]$ such that:
$$
g(\hat{q}) = \hat{q}^T[K]\hat{q}
$$

Where,

$$
[K] = \left(\begin{array}{cc} 
\sigma & Z^T \\
Z & S- \sigma I
\end{array}\right)
$$

and,

$\sigma = Trace[B]$  

$[S] = [B] + [B]^T$ 

$[Z] = [B_{23} - B_{32}, B_{31} - B_{13}, B_{12} - B_{21}]^T$

$[B] = \sum_{k=1}^N w_k{}^B\hat{v}_k{}^N\hat{v}_k^T$

The maximal eigenvalue and corresponding eihenfunction is determined from $[K]$, thus maximizing $g$.

In [60]:
def OuterProduct(Bv, Nv):
    """Outer Product BvNv^T:"""
#     B = np.array([[bv[0] * nv[0], bv[0] * nv[1], bv[0] * nv[2]],
#                   [bv[1] * nv[0], bv[1] * nv[1], bv[1] * nv[2]],
#                   [bv[2] * nv[0], bv[2] * nv[1], bv[2] * nv[2]]])
            
    """Using list comprehensions (I like list comprehensions)"""
    vvT = np.array([Bv[i]*Nv[j] for i in range(len(Bv)) for j in range(len(Nv))])
    return vvT.reshape(3,3)

def Bmatrix(BNv1, BNv2, w1, w2):
    return BNv1 * w1 + BNv2 * w2

In [85]:
B = Bmatrix(OuterProduct(B_v1, N_v1), OuterProduct(B_v2, N_v2), 1, 1)

In [224]:
def Kmatrix(B):
    sig = B.trace()  
    S = B + B.T
    Z = [B[1][2] - B[2][1], B[2][0] - B[0][2], B[0][1] - B[1][0]]
    SsI = S - sig*np.eye(3)
    
    return np.array([[ sig,      Z[0],      Z[1],      Z[2]],
                     [Z[0], SsI[0][0], SsI[0][1], SsI[0][2]],
                     [Z[1], SsI[1][0], SsI[1][1], SsI[1][2]],
                     [Z[2], SsI[2][0], SsI[2][1], SsI[2][2]]])

In [225]:
K = Kmatrix(B)

In [131]:
#Solving for the eigenvalues and corresponding eigenvectors.
eig, eigvec = np.linalg.eig(K)

#Determining the index corresponding to the maximum eigenvalue. (np returns the eigenvectors as column format)
idx, max_eig = np.argmax(eig), max(eig)
eig_vec = eigvec[:, idx]

print("Index: ", idx)
print("Max eigenvalue: ", max_eig)
print("Eigenvector: ", eig_vec)

Index:  3
Max eigenvalue:  1.9996751423328836
Eigenvector:  [ 0.02640542 -0.84099401  0.50198046 -0.20011858]


In [132]:
#The Eigenvector is in quaternion format, therefore, to acquire the corresponding DCM:
def Q_to_DCM(q):
    
    q_0, q_1, q_2, q_3 = q
    
    return np.array([[q_0**2 + q_1**2 - q_2**2 - q_3**2, 2 * (q_1 * q_2 + q_0 * q_3), 2 * (q_1 * q_3 - q_0 * q_2)],
                     [2 * (q_1 * q_2 - q_0 * q_3), q_0**2 - q_1**2 + q_2**2 - q_3**2, 2 * (q_2 * q_3 + q_0 * q_1)],
                     [2 * (q_1 * q_3 + q_0 * q_2), 2 * (q_2 * q_3 - q_0 * q_1), q_0**2 - q_1**2 - q_2**2 + q_3**2]],
                     dtype = np.float64)

In [133]:
print(Q_to_DCM(eig_vec))

[[ 0.41593634 -0.85489355  0.31008704]
 [-0.83375669 -0.49463674 -0.24532484]
 [ 0.36310707 -0.15649763 -0.91851061]]


## Quest.

The cost function $J$ is now rewritten as:

$$
J = \sum_{k = 1}^N w_k - g \equiv \sum_{k = 1}^N w_k - \lambda_{opt}
$$

Such that:
$$
\lambda_{opt} = \sum_{k = 1}^N w_k - J
$$

$$
\lambda_{opt} \approx \sum_{k = 1}^N w_k
$$

As $J << 1.$

The Newton-Raphson method is employed to obtain a more accurate value for $\lambda_{opt}$:
$$
\lambda_{max} = \lambda_{i} = \lambda_{i-1} - \frac{f(\lambda_{i-1})}{f'(\lambda_{i-1})}
$$

In [250]:
symbols = "s"

w1, w2 = 1.0, 2.0

#eig0 = w1 + w2

#f(s) = det([K]-s[I]), where f(s) corresponds to the eigensolutions.
def f(K):
    return det(K - s*eye(4)) 

#f'(s) = d/ds(det([K]-s[I])) which has a general form.
def dfds(f):
    return diff(f, s)
    
#eig[i] = eig[i-1] - f(eig[i-1])/f'(eig[i-1])   
def Newton_Raphson(f, dfds, eig0):
    
    for i in range(1,10):
        
        eig0 -= (f/dfds).evalf(subs={s:eig0})
        
    return np.float64(eig0)

f = f(K)
dfds = dfds(f)
eig_opt = Newton_Raphson(f, dfds, eig0)

Now solving for the CRP coordinate set via:

$$
([S] - \sigma[I])\hat{q} + [Z] = \lambda_{opt}\hat{q}
$$

$$
\hat{q} = \big((\lambda_{opt} + \sigma)[I] - [S]\big)^{-1}[Z]
$$

In [275]:
def CRP(B, e):
    
    sig = B.trace()  
    S = B + B.T
    Z = [B[1][2] - B[2][1], B[2][0] - B[0][2], B[0][1] - B[1][0]]

    return np.dot(np.linalg.inv((e + sig)*np.eye(3) - S), Z)

In [276]:
CRP = CRP(B, eig_opt)

In [278]:
#Converting to Quaternions:
q = (1 / np.sqrt(1 + np.dot(CRP, CRP)) * np.insert(CRP, 0, 1))
print(q)

[ 0.02640542 -0.84099401  0.50198046 -0.20011858]


In [279]:
#Mapping Quaternions to the corresponding DCM:
def Q_to_DCM(q):
    
    q_0, q_1, q_2, q_3 = q
    
    return np.array([[q_0**2 + q_1**2 - q_2**2 - q_3**2, 2 * (q_1 * q_2 + q_0 * q_3), 2 * (q_1 * q_3 - q_0 * q_2)],
                     [2 * (q_1 * q_2 - q_0 * q_3), q_0**2 - q_1**2 + q_2**2 - q_3**2, 2 * (q_2 * q_3 + q_0 * q_1)],
                     [2 * (q_1 * q_3 + q_0 * q_2), 2 * (q_2 * q_3 - q_0 * q_1), q_0**2 - q_1**2 - q_2**2 + q_3**2]],
                     dtype = np.float64)


In [280]:
print(Q_to_DCM(q))

[[ 0.41593634 -0.85489355  0.31008704]
 [-0.83375669 -0.49463674 -0.24532484]
 [ 0.36310707 -0.15649763 -0.91851061]]


## Optimal Linear Attitude Estimator - OLAE

From the Cayley transform:

$$
s_i = {}^B\hat{v_i} + {}^N\hat{v_i}
$$

$$
d_i = {}^B\hat{v_i} - {}^N\hat{v_i}
$$

Therefore:

$$
d_i = [\tilde{s_i}]\bar{q}
$$

Solving for $\bar{q}$:

$$
\bar{q} = \big([S]^T[W][S]\big)^{-1}[S]^T[W]d
$$

In [303]:
def s(Bv, Nv): return Bv + Nv

def d(Bv, Nv): return Bv - Nv

def skew(x): return np.array([[0, -x[2], x[1]],
                              [x[2], 0, -x[0]],
                              [-x[1], x[0], 0]])

def q_bar(S, W, d):
    STWS = np.linalg.inv(np.linalg.multi_dot((S.T, W, S)))
    STWd = np.linalg.multi_dot((S.T, W, d))
    return np.dot(STWS, STWd)

In [304]:
s1 = s(B_v1, N_v1)
s2 = s(B_v2, N_v2)

d1 = d(B_v1, N_v1)
d2 = d(B_v2, N_v2)

S = np.vstack((skew(s1), skew(s2)))
W = np.eye(6)
d = np.vstack((d1, d2)).reshape((6,1))

In [305]:
q_bar = q_bar(S, W, d).reshape((3,))

In [306]:
#Converting to quaternions:
q = (1 / np.sqrt(1 + np.dot(q_bar, q_bar)) * np.insert(q_bar, 0, 1))
print(q)

[ 0.0264126  -0.84107459  0.5018673  -0.20006281]


In [307]:
#Quaternion to DCM:
print(Q_to_DCM(q))

[[ 0.41620817 -0.85478402  0.31002425]
 [-0.83364731 -0.49486317 -0.24523989]
 [ 0.36304672 -0.15638004 -0.9185545 ]]
